In [26]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [27]:
file_dir = 'C:/Users/Vince/Desktop/UT Data Class/Module 8 ETL - Extract_Transform_Load'

f'{file_dir}filename'

'C:/Users/Vince/Desktop/UT Data Class/Module 8 ETL - Extract_Transform_Loadfilename'

In [28]:
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)


In [29]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie)
    
    return movie

In [33]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

from distutils.log import error
from sqlalchemy import false


def extract_transform_load():
    
    #Create the path to your file directory and variables for the three files. 
    #file_dir = 'C:/Users/Vince/Desktop/UT Data Class/Module 8 ETL - Extract_Transform_Load'

    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_file = pd.read_csv(f'{file_dir}/movies_metadata.csv', low_memory=false)
    ratings_file = pd.read_csv(f'{file_dir}/ratings.csv')

    # Open and read the Wikipedia data JSON file.
    with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
   
    # Write a list comprehension to filter out TV shows.
    # imdb_id is null       
    kaggle_file = [movie for movie in wiki_movies_raw
                    if ('Director' in movie or 'Directed by' in movie)
                    and 'imdb_link' in movie]
    len(kaggle_file)

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movie = [clean_movie(movie) for movie in wiki_movies_raw]

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movie)
    sorted(wiki_movies_df.columns.tolist())

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.   
   
    try:

        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        print(len(wiki_movies_df))
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        print(len(wiki_movies_df))
        wiki_movies_df.head()

    except: 

        print("An exception occured")

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    
    [[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

    # Create a variable that will hold the non-null values from the “Box office” column.

    box_office = wiki_movies_df['Box office'].dropna()

    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    
    box_office[box_office.map(lambda x: type(x) != str)]

    # Write a regular expression to match the six elements of "form_one" of the box office data.
   
    form_one = r'\$\d+\.?\d*\s*[mb]illion'

    box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

    # Write a regular expression to match the three elements of "form_two" of the box office data.
    
    form_two = r'\$\d{1,3}(?:,\d{3})+'
    
    box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

    # Add the parse_dollars function.
    
    def parse_dollar(s):
        if type(s) != str:
            return np.nan
        
    # Clean the box office column in the wiki_movies_df DataFrame.

    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollar(s))
    
    # Clean the budget column in the wiki_movies_df DataFrame.
    
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

    # Clean the release date column in the wiki_movies_df DataFrame.
    
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # Clean the running time column in the wiki_movies_df DataFrame.
    
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
     
    # 2. Clean the Kaggle metadata.

    kaggle_file.dtypes
    kaggle_file[~kaggle_file['adult'].isin(['True','False'])]
    kaggle_file = kaggle_file[kaggle_file['adult'] == 'False'].drop('adult',axis='columns')
    kaggle_file['video'] == 'True'
    kaggle_file['video'] = kaggle_file['video'] == 'True'

    kaggle_file['budget'] = kaggle_file['budget'].astype(int)
    kaggle_file['id'] = pd.to_numeric(kaggle_file['id'], errors='raise')
    kaggle_file['popularity'] = pd.to_numeric(kaggle_file['popularity'], errors='raise')

    kaggle_file['release_date'] = pd.to_datetime(kaggle_file['release_date'])

    # 3. Merged the two DataFrames into the movies DataFrame.

    movies_df = pd.merge(wiki_movies_df, kaggle_file, on='imdb_id', suffixes=['_wiki','_kaggle'])

    # 4. Drop unnecessary columns from the merged DataFrame.

    movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)

    # 5. Add in the function to fill in the missing Kaggle data.

    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
            , axis=1)

        df.drop(columns=wiki+column, inplace=True)

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.

    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
    movies_df

    # 7. Filter the movies DataFrame for specific columns.

    for col in movies_df.columns:
        lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
        value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
        num_values = len(value_counts)
    if num_values == 1:
        print(col)

    # 8. Rename the columns in the movies DataFrame.

    movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)

    # 9. Transform and merge the ratings DataFrame.

    rating_counts = ratings_file.groupby(['movieId','rating'], as_index=False).count()
    rating_counts = ratings_file.groupby(['movieId','rating'], as_index=False).count() \
        .rename({'userId':'count'}, axis=1)
    
    rating_counts = ratings_file.groupby(['movieId','rating'], as_index=False).count() \
        .rename({'userId':'count'}, axis=1) \
        .pivot(index='movieId',columns='rating', values='count')

    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')

    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
    
    return wiki_movies_df, movies_with_ratings_df, movies_df


In [34]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = '../Resources'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [35]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

FileNotFoundError: [Errno 2] No such file or directory: '../Resources/movies_metadata.csv.csv'

In [36]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [ ]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df

In [ ]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df

In [ ]:
# 15. Check the movies_df DataFrame. 
movies_df